***3-Layer CNN Model Approach***

This uses a basic input of [block_size] x [block_size] square of pixels from sentinel 2 imagery to predict land cover classes.
It is trained using the 30m resolution land cover class dataset from USFS.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import folium
from pathlib import Path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
root_dir = Path(notebook_dir).parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))
from src.models.predictions import load_model, predict_land_cover
from src.models.cnn_model import LandCoverCNN
from src.data.CNN.cnn_data_generator import generate_target_pixels, generate_s2_pixel_blocks, find_matching_lc_pixel_classes
s2_data_folder = os.path.join(root_dir, 'data', 'raw', 'sentinel2_imagery')
lc_data_folder = os.path.join(root_dir, 'data', 'raw', 'USFS_land_cover')
rgb_model_path = os.path.join(root_dir, 'src', 'models', 'rgb_80x80_CNN_block_model.pth')
nir_model_path = os.path.join(root_dir, 'src', 'models', 'rgb_nir_80x80_CNN_block_model.pth')
classes = [
    "Trees",
    "Tall Shrubs & Trees Mix (SEAK Only)", 
    "Shrubs & Trees Mix",
    "Grass/Forb/Herb & Trees Mix",
    "Barren & Trees Mix",
    "Tall Shrubs (SEAK Only)",
    "Shrubs",
    "Grass/Forb/Herb & Shrubs Mix",
    "Barren & Shrubs Mix",
    "Grass/Forb/Herb",
    "Barren & Grass/Forb/Herb Mix",
    "Barren or Impervious",
    "Snow or Ice",
    "Water",
    "Non-Processing Area Mask"
]

# Parameters
roi = "boulder"
sample_size = 500
block_size = 80 # Multiples of 8 for this
s2_tif = os.path.join(s2_data_folder, f'sentinel2_10m_{roi}_2021-07.tif')
lc_tif = os.path.join(lc_data_folder, f'landcover_30m_{roi}_2021.tif')

# Generate target pixels, s2 blocks, and USFS classes
target_pixels = generate_target_pixels(s2_tif, sample_size, block_size)
s2_blocks, valid_target_pixels = generate_s2_pixel_blocks(s2_tif, target_pixels, block_size)
usfs_classes, utms, albers, lat_lons = find_matching_lc_pixel_classes(lc_tif, s2_tif, valid_target_pixels)
print(f"s2_blocks len: {len(s2_blocks)}")
print(f"usfs_classes len: {len(usfs_classes)}")
if len(s2_blocks) != len(usfs_classes):
    raise SystemExit("Differing lens of s2 blocks and usfs classes lists")

# Run the target pixel blocks through the model
nir_model = load_model(nir_model_path)
nir_model_outputs, nir_difs = [], 0
for block in s2_blocks:
    nir_model_outputs.append(predict_land_cover(nir_model, block))
for i in range(len(nir_model_outputs)):
    if int(usfs_classes[i]) != int(nir_model_outputs[i]):
        nir_difs += 1

print(f"% of pixels different in nir outputs: %{(nir_difs/len(usfs_classes))*100}")

# Visualize the target pixels
m = folium.Map(location=(lat_lons[0][1], lat_lons[0][0]), zoom_start=14)

for i in range(len(nir_model_outputs)):
    if int(usfs_classes[i]) == int(nir_model_outputs[i]):
        folium.Marker(
            location=[lat_lons[i][1], lat_lons[i][0]],
            tooltip=f"""
            USFS class: {usfs_classes[i]} <br>({classes[usfs_classes[i]]}), <br>
            nir model output: {nir_model_outputs[i]} <br>
            ({classes[nir_model_outputs[i]]})
            """,
            icon=folium.Icon(color="green")
        ).add_to(m)
    else:
        folium.Marker(
            location=[lat_lons[i][1], lat_lons[i][0]],
            tooltip=f"""
            USFS class: {usfs_classes[i]} <br>({classes[usfs_classes[i]]}), <br>
            nir model output: {nir_model_outputs[i]} <br>
            ({classes[nir_model_outputs[i]]})
            """,
            icon=folium.Icon(color="red")
        ).add_to(m)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
    ).add_to(m)

folium.LayerControl().add_to(m)
m

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
s2_blocks len: 500
usfs_classes len: 500
Block size: 80×80
Feature map after pooling: 128 × 10 × 10
Flattened features: 12800
% of pixels different in nir outputs: %35.6
